In [1]:
from geopy.distance import geodesic
import osmium
import os
import math
from scipy.spatial import KDTree
import numpy as np
import random

out_file = "../osm/buildings.osm"
out_points = "../osm/points.txt"
input_file = '../osm/export.osm'
point_count = 100000
SAVE_MODIFIED_WAYS=True

In [2]:
class OSMHandler(osmium.SimpleHandler):
    def __init__(self):
        super().__init__()
        self.node_locs = osmium.index.create_map("sparse_mem_array")
        self.loc_handler = osmium.NodeLocationsForWays(self.node_locs)
        self.ways = []
        self.builsings = []
        self.nods_way = []
        self.nods_build = []
        
    def node(self, n: osmium.Node):
        self.node_locs.set(n.id, n.location)

    def way(self, w: osmium.Way):
       
        if 'highway' in w.tags:
            total_length = 0.0
            lanes = float(w.tags.get('lanes', '2'))
            prev_node = None
            for n in w.nodes:
                self.nods_way.append((n.ref, n.location, 0, 0))
 
        elif 'building' in w.tags:
            lat = 0
            lon = 0
            for n in w.nodes:
                self.nods_build.append((n.ref, n.location))
                lat = lat + n.lat
                lon = lon + n.lon
            lat = lat/len(w.nodes)
            lon = lon/len(w.nodes)
            self.builsings.append((w.id, dict(w.tags), list(w.nodes), lat, lon))


In [3]:
def save_start_end_predict(buildings, nodes_way):
    # Создаем массив координат нод для KD-Tree
    node_coords = [(loc.lat, loc.lon) for (ref, loc, r_cap, c_cap) in nodes_way]
   
    node_tree = KDTree(node_coords)

    capacity_residential = 1.0 
    capacity_commercial = 10.0 
    
    points_from_dict = {}
    points_to_dict = {}

    for id, tags, nodes, lat, lon in buildings:
        building_type = tags.get('building', "yes")
        building_coord = (lat, lon)
        capacity = 5

        distance, index = node_tree.query(building_coord)
        
        if building_type in ['residential', 'house', 'apartments', 'dormitory', 'terrace', 'detached', 'bungalow']:
            match building_type:
                case 'apartments':
                    # print(tags.get('building:levels', -1), tags.get('length', -1), tags.get('width', -1) )
                    capacity = 10
                case 'residential','residential', 'house','dormitory', 'terrace', 'detached', 'bungalow':
                    # print("Обработка случая 2")
                    capacity = 5    
            ref, loc, r_cap, c_cap = nodes_way[index]
            nodes_way[index] = (ref, loc, r_cap + capacity, c_cap)
            points_from_dict[ref] = points_from_dict.get(ref, 0) + capacity
        elif building_type in ['commercial', 'office', 'industrial', "retail", "store", "school",
                                "hospital", 'garages', 'warehouse', 'service', 'kindergarten']:
            capacity = capacity_commercial
            ref, loc, r_cap, c_cap = nodes_way[index]
            nodes_way[index] = (ref, loc, r_cap, c_cap + capacity)
            points_to_dict[ref] = points_to_dict.get(ref, 0) + capacity
        else: 
            #считаю дом промежуточным если ниодин тег не подходит
            capacity = 1
            ref, loc, r_cap, c_cap = nodes_way[index]
            nodes_way[index] = (ref, loc, r_cap, c_cap + capacity)
            points_to_dict[ref] = points_to_dict.get(ref, 0) + capacity
            ref, loc, r_cap, c_cap = nodes_way[index]
            nodes_way[index] = (ref, loc, r_cap + capacity, c_cap)
            points_from_dict[ref] = points_from_dict.get(ref, 0) + capacity

    points_from = list(points_from_dict.keys())
    weights_from = list(points_from_dict.values())

    points_to = list(points_to_dict.keys())
    weights_to = list(points_to_dict.values())
    print("Point go")
    with open(out_points, 'w') as file:
        for i in range(point_count):
            selected_point_from = random.choices(points_from, weights=weights_from, k=1)[0]
            selected_point_to = random.choices(points_to, weights=weights_to, k=1)[0]
            # Записываете числа в файл, разделяя их символом '/'
            file.write(f"{selected_point_from} {selected_point_to}\n")
            if (i*10%point_count == 0):
                print(f"points: {i}")

    print("Point end")


In [4]:
def save_modified_ways(building, nodes_building, nodes_way, writer):

    for id, loc, r_cap, c_cap in nodes_way:
        node = osmium.osm.mutable.Node()
        node.id = id
        node.location = loc 
        node.tags = {}
        if (r_cap != 0):
            node.tags["r_cap"] = str(r_cap)
        if (c_cap != 0):
            node.tags["c_cap"] = str(c_cap) 
        if (r_cap != 0 or c_cap != 0):
            writer.add_node(node)
        
    for id, loc in nodes_building:
        node = osmium.osm.mutable.Node()
        node.id = id
        node.location = loc 
        writer.add_node(node)

    for way_id, tags, nods, lat, lon in building:
        new_way = osmium.osm.mutable.Way()
        new_way.id = way_id
        new_way.tags = tags
        new_way.nodes = nods
        writer.add_way(new_way)

In [5]:
h = OSMHandler()
h.apply_file(input_file, locations=True)
save_start_end_predict(h.builsings, h.nods_way)
if SAVE_MODIFIED_WAYS:
    #Демонстративный вывод домиков
    if os.path.exists(out_file):
        os.remove(out_file)
    try:
        writer = osmium.SimpleWriter(out_file)
        save_modified_ways(h.builsings, h.nods_build, h.nods_way, writer)
    finally:
        writer.close() 

Point go
points: 0
points: 1000
points: 2000
points: 3000
points: 4000
points: 5000
points: 6000
points: 7000
points: 8000
points: 9000
Point end
